# Text Summarization

We are going to create a summarizer which will fetch data from different online articles say Wikipedia it will passs those articles and convert them into mere strings and then it will summazrize all of those articles.

Wikipedia articles are huge, they are like 2-3 different pages and it will summarize all those artocles into 3-5 sentences

To find summary for an articles there are two approaches
- A simple Natural Language Processing based approach
- A deep NLP based approach (more in demand and research still going on)

#### Sample paragraph
Thank you all so very much. Thank you to the Academy. Thank You to all of you in this room. I have to congratulate the other incredible
nominess this year. The Revenant was the product of the tireless efforts of an unbelievable cast and crew.

#### Tokenization
- Thank you all so very much.
- Thank you to the Academy.
- Thank You to all of you in this room.
- I have to congratulate the other incredible nominess this year.
- The Revenant was the product of the tireless efforts of an unbelievable cast and crew.

#### Preprocessed
- thank you all so very much.
- thank you to the academy.
- thank you to all of you in this room.
- i have to congratulate the other incredible nominess this year.
- the revenant was the product of the tireless efforts of an unbelievable cast and crew.

#### Create the histogram
#### Weighted histogram
#### Sentence Scores
#### Sort by score
#### Pickle N largest
#### Sentence summary
The Revenant was the product of the tireless efforts of an unbelievable cast and crew.(Highest sentence score)
Thank You to all of you in this room.(2nd highest sentence score)

# Fetching article data from the web

### Creating an Article Summarizer

In [31]:
import bs4 as bs
import urllib.request
import re
import nltk
nltk.download('stopwords')
import heapq

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\SHADAAB\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [3]:
# Getting the data
source = urllib.request.urlopen('https://en.wikipedia.org/wiki/Global_warming').read()

In [5]:
 soup = bs.BeautifulSoup(source,'lxml')          #lxml is a type of parser that beautiful soup uses to parser a html document

We have fetched from the wikipedia global warming article source and we have also parsed that article using this soup object ,using beautiful library

### Parsing the data using Beautiful Soup

Here we are going to get string data and that is the data that we are going to use to find the summary

Here we are finding all the text that are in the paragraph tag in this HTML. And in the wikipedia whatever texts we see corresponding to an article is always inside of a paragraph tag. 
So we can fetch all the text from the paragraph tags so that we will get all the text from the whole article

In [7]:
text = ""
for paragraph in soup.find_all('p'):  #this p is the html p tag which is paragraph tag
    text += paragraph.text
# this append is only for lists not for string #earlier cose was text.append(paragraph)

This whole text contains each and every line from the whole global warming article on wikipedia right.

##### Important
- Here we are getting all the data from the p tag beacuse wikipedia follows this style of writing. They always put all the articles inside of p tag so that we can just find all the tags
- But there are many sites which put the articles inside of the 'div' tag or inside of a 'span' tag.

# Preprocessing the text

In [12]:
text = re.sub(r'\[[0-9]*\]',' ',text)
text

'\n\nGlobal warming is a long-term rise in the average temperature of the Earth\'s climate system, an aspect of climate change shown by temperature measurements and by multiple effects of the warming.   Though earlier geological periods also experienced episodes of warming,  the term commonly refers to the observed and continuing increase in average air and ocean temperatures since 1900 caused mainly by emissions of greenhouse gases in the modern industrial economy.  In the modern context the terms global warming and climate change are commonly used interchangeably,  but climate change includes both global warming and its effects, such as changes to precipitation and impacts that differ by region.   Many of the observed changes in climate since the 1950s are unprecedented in the instrumental temperature record, and in historical and paleoclimate proxy records of climate change over thousands to millions of years. \nIn 2013, the Intergovernmental Panel on Climate Change (IPCC) Fifth Ass

In [13]:
text = re.sub(r'\s+',' ',text)

In [14]:
clean_text = text.lower()
clean_text = re.sub(r'\W',' ',clean_text)
clean_text = re.sub(r'\d',' ',clean_text)
clean_text = re.sub(r'\s+',' ',clean_text)

We have 'clean_text' and main global warming 'text'. In clean_text we have no punctuations no extra spaces, ready to form a model.

The reason for building this clean_text apart from this text is that whanever we are building a histogram, then we are going to build this histogram out of this text (clean_text).
But there are some numbers as 42 C or temperature those are important. Those are not important for building histogram but are important for the summary.

- Clean_text is for creating the histogram
- text is for creating the summary

# Tokenizing Article into sentence

In [17]:
sentences = nltk.sent_tokenize(text)  #because this text contains temperature etc informations. clean text contains only plain text
sentences

[" Global warming is a long-term rise in the average temperature of the Earth's climate system, an aspect of climate change shown by temperature measurements and by multiple effects of the warming.",
 'Though earlier geological periods also experienced episodes of warming, the term commonly refers to the observed and continuing increase in average air and ocean temperatures since 1900 caused mainly by emissions of greenhouse gases in the modern industrial economy.',
 'In the modern context the terms global warming and climate change are commonly used interchangeably, but climate change includes both global warming and its effects, such as changes to precipitation and impacts that differ by region.',
 'Many of the observed changes in climate since the 1950s are unprecedented in the instrumental temperature record, and in historical and paleoclimate proxy records of climate change over thousands to millions of years.',
 'In 2013, the Intergovernmental Panel on Climate Change (IPCC) Fifth

In [22]:
stop_words = nltk.corpus.stopwords.words('english')

In [23]:
stop_words

['i',
 'me',
 'my',
 'myself',
 'we',
 'our',
 'ours',
 'ourselves',
 'you',
 "you're",
 "you've",
 "you'll",
 "you'd",
 'your',
 'yours',
 'yourself',
 'yourselves',
 'he',
 'him',
 'his',
 'himself',
 'she',
 "she's",
 'her',
 'hers',
 'herself',
 'it',
 "it's",
 'its',
 'itself',
 'they',
 'them',
 'their',
 'theirs',
 'themselves',
 'what',
 'which',
 'who',
 'whom',
 'this',
 'that',
 "that'll",
 'these',
 'those',
 'am',
 'is',
 'are',
 'was',
 'were',
 'be',
 'been',
 'being',
 'have',
 'has',
 'had',
 'having',
 'do',
 'does',
 'did',
 'doing',
 'a',
 'an',
 'the',
 'and',
 'but',
 'if',
 'or',
 'because',
 'as',
 'until',
 'while',
 'of',
 'at',
 'by',
 'for',
 'with',
 'about',
 'against',
 'between',
 'into',
 'through',
 'during',
 'before',
 'after',
 'above',
 'below',
 'to',
 'from',
 'up',
 'down',
 'in',
 'out',
 'on',
 'off',
 'over',
 'under',
 'again',
 'further',
 'then',
 'once',
 'here',
 'there',
 'when',
 'where',
 'why',
 'how',
 'all',
 'any',
 'both',
 'each

Whenever we are creating a word histogram then we really don't want to consider stopwords, we only wants to consider important words

# Building the histogram

In [27]:
# Basic Histogram
word2count = {}
for word in nltk.word_tokenize(clean_text):
    if word not in stop_words:
        if word not in word2count.keys():
            word2count[word] = 1
        else:
            word2count[word] += 1

In [28]:
# Weighted histogram
# To create a weighted histogram, we have to divide all the values corresponding to the words by the maximum value
for key in word2count.keys():
    word2count[key] = word2count[key]/max(word2count.values())

In [29]:
word2count

{'emanate': 0.009900990099009901,
 'academy': 0.019801980198019802,
 'pollutants': 0.009900990099009901,
 'risk': 0.04950495049504951,
 'fish': 0.009900990099009901,
 'management': 0.019801980198019802,
 'largest': 0.019801980198019802,
 'resulted': 0.009900990099009901,
 'el': 0.019801980198019802,
 'loa': 0.009900990099009901,
 'injury': 0.009900990099009901,
 'agreement': 0.07920792079207921,
 'americans': 0.019801980198019802,
 'although': 0.039603960396039604,
 'pronounced': 0.039603960396039604,
 'five': 0.009900990099009901,
 'impact': 0.039603960396039604,
 'greater': 0.009900990099009901,
 'currents': 0.009900990099009901,
 'royal': 0.019801980198019802,
 'known': 0.0594059405940594,
 'produces': 0.019801980198019802,
 'shifting': 0.009900990099009901,
 'mitigation': 0.07920792079207921,
 'patterns': 0.019801980198019802,
 'roughly': 0.009900990099009901,
 'benefits': 0.0297029702970297,
 'cycle': 0.0594059405940594,
 'uneven': 0.009900990099009901,
 'ar': 0.009900990099009901

# Calculating the sentence scores

In [30]:
sent2score = {}
for sentence in sentences:
    for word in nltk.word_tokenize(sentence.lower()):
        if word in word2count.keys():
            if len(sentence.split(' ')) < 25:
                if sentence not in sent2score.keys():
                    sent2score[sentence] = word2count[word]
                else:
                    sent2score[sentence] += word2count[word]
print(sent2score)

{'Public reactions to global warming and concern about its effects have been increasing, while many perceiving is as the worst global threat.': 3.911061795834756, 'In most scenarios, emissions continue to rise over the century, while in a few, emissions are reduced.': 1.421873221804939, 'The Framework Convention was agreed on in 1992, but global emissions have risen since then.': 1.9433550551456946, 'Solid and liquid particles known as aerosols – from volcanoes, plankton and human-made pollutants – reflect incoming sunlight, cooling the climate.': 1.4975532035962216, 'The agreement replaced the Kyoto protocol.': 0.16831683168316833, "In addition to their direct effect by scattering and absorbing solar radiation, aerosols have indirect effects on the Earth's radiation budget.": 1.0839877091157393, 'Sulfate aerosols act as cloud condensation nuclei and thus lead to clouds that have more and smaller cloud droplets.': 0.4653465346534653, 'With this information, scientists can produce scena

# Getting the summary


In [33]:
best_sentences = heapq.nlargest(25,sent2score,key = sent2score.get)   # we are selecting 25 best sentences from the whole articels
print('------------------------------------')
for sentence in best_sentences:
    print(sentence)

------------------------------------
Possible societal responses to global warming include mitigation by emissions reduction, adaptation to its effects, and possible future climate engineering.
Global warming refers to global averages, with the amount of warming varying by region.
According to basic physical principles, the greenhouse effect produces warming of the lower atmosphere (the troposphere), but cooling of the upper atmosphere (the stratosphere).
Public reactions to global warming and concern about its effects have been increasing, while many perceiving is as the worst global threat.
Mitigation of and adaptation to climate change are two complementary responses to global warming.
Climate change adaptation is the process of adjusting to actual or expected climate change and its effects.
In the late 19th century, scientists first argued that human emissions of greenhouse gases could change the climate.
Global warming of around 4 °C relative to late 20th century levels could pose